# BAX 452 Assignment 4
### Group 10: Yi Yin (Ian) Chen/Priyanka Malhotra/Sushma Niveni Pindiga

### Exploratory Data Analysis (EDA) (20 pts):
### Conduct a thorough exploration of the provided datasets. This includes understanding the structure, identifying missing values, and summarizing key statistics. Generate visualizations to understand the distribution of important variables, relationships between different variables, and any other patterns or anomalies in the data such as peculiar patterns or outliers.

In [2]:
## Import necessary packages
import pandas as pd
import py7zr
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegressionCV
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

In [3]:
## Import the two datasets and merge
# Path to your .7z file
file_path = 'longyear-obese-hypertensive-40-57-medical-SMALL-sample.7z'
# Open the .7z file
with py7zr.SevenZipFile(file_path, mode='r') as z:
    all_files = z.getnames()
    # Assuming there's only one CSV file in the .7z archive
    csv_file_name = all_files[0]
    # Extract the CSV file to memory
    csv_file_dict = z.read([csv_file_name])
    csv_file = csv_file_dict[csv_file_name]

# Load the CSV data into a pandas DataFrame
raw_df = pd.read_csv(csv_file)

# Now you can work with the DataFrame `df` as usual
file_path = 'longyear-obese-hypertensive-40-75-scripts-SMALL-sample.7z'
# Open the .7z file
with py7zr.SevenZipFile(file_path, mode='r') as z:
    all_files = z.getnames()
    # Assuming there's only one CSV file in the .7z archive
    csv_file_name = all_files[0]
    # Extract the CSV file to memory
    csv_file_dict = z.read([csv_file_name])
    csv_file = csv_file_dict[csv_file_name]

# Load the CSV data into a pandas DataFrame
raw_df2 = pd.read_csv(csv_file)

/var/folders/9_/v2c7033x6k1b1rzd02qw1x280000gn/T/ipykernel_9296/1667547875.py:28: DtypeWarning: Columns (9,10,29,43) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df2 = pd.read_csv(csv_file)


In [5]:
# Analyze columns
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623525 entries, 0 to 623524
Data columns (total 34 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   journey_id         623525 non-null  object 
 1   episode_id         623525 non-null  object 
 2   visit_id           621650 non-null  object 
 3   encounter_id       623525 non-null  object 
 4   claim_date         623525 non-null  object 
 5   patient_state      623525 non-null  object 
 6   patient_short_zip  623512 non-null  float64
 7   patient_age        616980 non-null  float64
 8   patient_gender     623393 non-null  object 
 9   place_of_service   484555 non-null  object 
 10  visit_type         600454 non-null  object 
 11  payor              608936 non-null  object 
 12  payor_channel      586086 non-null  object 
 13  ref_npi            297501 non-null  float64
 14  hcp_npi            434653 non-null  float64
 15  hcp_taxonomy       428209 non-null  object 
 16  hc

In [6]:
raw_df.head()

,journey_id,episode_id,visit_id,encounter_id,claim_date,patient_state,patient_short_zip,patient_age,patient_gender,place_of_service,...,diag_4,diag_5,rev_center_code,rev_center_units,proc_code,proc_modifier,proc_units,line_charge,claim_charge,smart_allowed
0,00006390c96ebfffed580074c35a16a7,d5e15811af1d772f54af52f3560be0a5,f22b7ab885c44afae5449d74f8498f53aea1b2cfa79598...,d5a7dc61071d977cef9ff762973d360f4bcf15a19790d9...,2020-07-13,CA,922.0,55.0,M,Home,...,NaN,NaN,NaN,1.0,A6549,NU,2.0,300.00,300.00,974.13
1,00006390c96ebfffed580074c35a16a7,d5e15811af1d772f54af52f3560be0a5,f22b7ab885c44afae5449d74f8498f53aea1b2cfa79598...,d5a7dc61071d977cef9ff762973d360f4bcf15a19790d9...,2020-07-13,CA,922.0,55.0,M,Home,...,NaN,NaN,NaN,1.0,A6549,NU,2.0,300.00,300.00,974.13
2,00006390c96ebfffed580074c35a16a7,e25e5285e4f66d35c9840bf4e040fe73,3f45ba8ebef02b0c09b09e864e57ba5ee9d6bc9acf1c39...,c60da949b58f0351c8781c062f65d92bce76d9532d1632...,2023-07-19,CA,922.0,60.0,M,Office,...,L03115,L03116,NaN,1.0,3074F,NaN,1.0,0.00,0.00,46.57
3,00006390c96ebfffed580074c35a16a7,e25e5285e4f66d35c9840bf4e040fe73,3f45ba8ebef02b0c09b09e864e57ba5ee9d6bc9acf1c39...,c60da949b58f0351c8781c062f65d92bce76d9532d1632...,2023-07-19,CA,922.0,60.0,M,Office,...,L03115,L03116,NaN,1.0,3078F,NaN,1.0,0.00,0.00,56.38
4,00006d564a4718c744f1bbf1b6c5f8c3,42ba4419d9ad7c9293d76686fb608f10,9e4534892128401c693a6ab2ea758cd6a0a6484972168b...,da5cef18317c14180ed561a84001d8a033d724d541a2c0...,2023-08-11,CA,900.0,55.0,F,Independent Laboratory,...,NaN,NaN,NaN,1.0,80053,NaN,1.0,88.07,88.07,15.16


In [8]:
raw_df.describe()

,patient_short_zip,patient_age,ref_npi,hcp_npi,hco_npi,rev_center_code,rev_center_units,proc_units,line_charge,claim_charge,smart_allowed
count,623512.000000,616980.000000,2.975010e+05,4.346530e+05,6.177890e+05,113926.000000,621558.000000,621331.000000,623525.000000,623525.000000,623525.000000
mean,907.514954,59.416910,1.502083e+09,1.508734e+09,1.485095e+09,456.083449,1.243236,4.319060,482.437268,482.437268,190.295642
std,40.034735,9.946158,2.868359e+08,2.825083e+08,2.859965e+08,451.591644,2.458118,55.208116,3582.592825,3582.592825,509.339625
min,74.000000,30.000000,1.003001e+09,1.003001e+09,1.003006e+09,1.000000,-14.000000,0.000000,0.000000,0.000000,0.000000
25%,900.000000,55.000000,1.255437e+09,1.265668e+09,1.245340e+09,301.000000,1.000000,1.000000,10.000000,10.000000,18.190000
50%,911.000000,60.000000,1.508952e+09,1.528016e+09,1.467631e+09,320.000000,1.000000,1.000000,70.000000,70.000000,75.230000
75%,922.000000,65.000000,1.740350e+09,1.740376e+09,1.720295e+09,521.000000,1.000000,1.000000,200.000000,200.000000,182.360000
max,999.000000,80.000000,1.992999e+09,1.993000e+09,1.992995e+09,3103.000000,730.000000,12473.000000,778750.000000,778750.000000,45035.960000


In [10]:
min(raw_df['claim_date'])

'2019-01-05'

In [11]:
max(raw_df['claim_date'])

'2023-09-27'

In [13]:
duplicate_counts = raw_df.duplicated(subset='journey_id', keep=False).sum()
print(duplicate_counts)

621110


In [14]:
duplicate_counts = raw_df2.duplicated(subset='journey_id', keep=False).sum()
print(duplicate_counts)

893567


In [7]:
raw_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894623 entries, 0 to 894622
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   journey_id                     894623 non-null  object 
 1   cohort_id                      894623 non-null  int64  
 2   claim_id                       894623 non-null  object 
 3   patient_gender                 894623 non-null  object 
 4   patient_state                  894623 non-null  object 
 5   patient_zip                    894501 non-null  float64
 6   patient_dob                    894619 non-null  object 
 7   pharmacy_npi                   839958 non-null  float64
 8   pharmacist_npi                 21578 non-null   float64
 9   prescriber_npi                 891155 non-null  object 
 10  primary_care_npi               3569 non-null    object 
 11  group_id                       273621 non-null  object 
 12  date_of_service               

In [7]:
raw_df2.head()

,journey_id,cohort_id,claim_id,patient_gender,patient_state,patient_zip,patient_dob,pharmacy_npi,pharmacist_npi,prescriber_npi,...,bin,plan_pay,reject_code_1,reject_code_3,reject_code_4,reject_code_5,ndc,active,start_date,end_date
0,00006390c96ebfffed580074c35a16a7,16,60618bd8d37311da702bf9349e76f8c26d515c082e136f...,M,CA,922.0,1961-01-01,1.215361e+09,NaN,1750479168,...,22659.0,NaN,NaN,NaN,NaN,NaN,47335067381,True,190001,NaN
1,00006390c96ebfffed580074c35a16a7,16,dbeb47defd161331e43f4e359e8d43e4ef7546ff58359b...,M,CA,922.0,1961-01-01,1.215361e+09,NaN,1750479168,...,22659.0,NaN,NaN,NaN,NaN,NaN,47335067381,True,190001,NaN
2,00006390c96ebfffed580074c35a16a7,16,02927d081fad10afffcba79bad27f94ff52c98e968aaec...,M,CA,922.0,1961-01-01,1.215361e+09,NaN,1750479168,...,22659.0,NaN,NaN,NaN,NaN,NaN,47335067381,True,190001,NaN
3,00006390c96ebfffed580074c35a16a7,16,a412c986e2503a5140ed1482e88b5ec50feeedaf2d89ec...,M,CA,922.0,1961-01-01,1.215361e+09,NaN,1750479168,...,22659.0,NaN,NaN,NaN,NaN,NaN,47335067381,True,190001,NaN
4,00006390c96ebfffed580074c35a16a7,16,b65634edd78998753fe9606b142b2fff7765d6c576973c...,M,CA,922.0,1961-01-01,1.215361e+09,NaN,1750479168,...,22659.0,NaN,NaN,NaN,NaN,NaN,65862059805,True,190001,NaN


In [9]:
# 1. Select 'diag_' columns
diag_columns = [col for col in raw_df.columns if col.startswith('diag_')]
df_diag = raw_df[diag_columns]

# 2. Combine values 
combined_list = df_diag.values.flatten().tolist()
combined_list = [x for x in combined_list if pd.notna(x)]

# 3. Process first three digits (same as before)
first_three_counts = {}
for diag_code in combined_list:
    first_three = diag_code[:3]
    first_three_counts[first_three] = first_three_counts.get(first_three, 0) + 1

In [10]:
import operator
# Sort the dictionary items by count (descending order)
sorted_counts = sorted(first_three_counts.items(), key=operator.itemgetter(1), reverse=True)

for items in sorted_counts:
    print(items)

('E11', 233629)
('I10', 199537)
('E78', 113939)
('Z00', 95083)
('E66', 69689)
('Z68', 54013)
('M25', 51750)
('M54', 50447)
('Z12', 32396)
('N18', 31950)
('Z01', 30622)
('Z13', 29844)
('M79', 29497)
('G47', 28339)
('R10', 27659)
('Z79', 27149)
('E55', 27085)
('Z71', 26523)
('R07', 25175)
('E03', 24910)
('R73', 24776)
('N39', 22340)
('M17', 21592)
('D64', 19973)
('I25', 17483)
('Z20', 16871)
('R53', 16402)
('I50', 15359)
('R06', 15340)
('J44', 14592)
('F41', 13833)
('Z23', 13598)
('I48', 12605)
('K21', 12366)
('G89', 11753)
('Z11', 11720)
('I11', 11344)
('F33', 10927)
('N40', 10720)
('J45', 10699)
('E87', 10668)
('M19', 10218)
('U07', 10043)
('M47', 9806)
('R68', 9479)
('A41', 9416)
('D50', 9387)
('J96', 9330)
('R05', 9106)
('M51', 8942)
('F32', 8895)
('I12', 8183)
('L03', 7899)
('B35', 7878)
('R79', 7617)
('Z51', 7527)
('R51', 7323)
('K76', 7175)
('Z76', 7142)
('R42', 7133)
('N17', 7130)
('H25', 7047)
('L97', 7010)
('M48', 6829)
('M62', 6795)
('R00', 6751)
('F17', 6672)
('C50', 6663)
('

### Insights from raw medical records
### Top 10

E11  Type 2 diabetes mellitus

I10  Essential (primary) hypertension

E78  Disorders of lipoprotein metabolism and other lipidemias

Z00  Encounter for general examination without complaint, suspected or reported diagnosis

E66  Overweight and obesity

Z68  Body mass index [BMI]

M25  Other joint disorder, not elsewhere classified

M54  Dorsalgia

Z12  Encounter for screening for malignant neoplasms

N18  Chronic kidney disease (CKD)


- The top 10 diagnosis from the raw medical dataset coincides with the main use of Ozempic, Type 2 diabetes, hypertension(lowering blood pressure), Overweight and BMI diagnosis.

- Some other top diagnosis includes CKD, Dorsolgia general examinations.



In [11]:
# Merge the datasets on jouney_id
merged_raw_df = pd.merge(raw_df, raw_df2, on='journey_id', how='inner')

In [5]:
# Check the merged data
merged_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29969378 entries, 0 to 29969377
Data columns (total 81 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   journey_id                     object 
 1   episode_id                     object 
 2   visit_id                       object 
 3   encounter_id                   object 
 4   claim_date                     object 
 5   patient_state_x                object 
 6   patient_short_zip              float64
 7   patient_age                    float64
 8   patient_gender_x               object 
 9   place_of_service_x             object 
 10  visit_type                     object 
 11  payor                          object 
 12  payor_channel                  object 
 13  ref_npi                        float64
 14  hcp_npi                        float64
 15  hcp_taxonomy                   object 
 16  hcp_specialty                  object 
 17  hco_npi                        float64
 18  